<a href="https://colab.research.google.com/github/TonioFlag/ProyectoFinalColas/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pingouin

In [125]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import pingouin as pg
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

In [3]:
arribo=pd.read_excel('/content/drive/MyDrive/Actual/COLAS/Tasas de arribo.xlsx')

num=[]
for i in arribo['dia']:
  if i=="Lunes":
    num.append(1)
  elif i=="Martes":
    num.append(2)
  elif i=="Miercoles":
    num.append(3)
  elif i=="Jueves":
    num.append(4)
  elif i=="Sabado":
    num.append(6)
  elif i=="Viernes":
    num.append(5)
  elif i=="Domingo":
    num.append(7)

arribo['num_dia']=num

intervalos=5
amplitud=arribo.min_cliente.max()-arribo.min_cliente.min()
intervalo=amplitud/5
clasificacion=[]
for i in arribo.min_cliente:
  if i <= intervalo: clasificacion.append(1)
  elif i <= 2*intervalo: clasificacion.append(2)
  elif i <= 3*intervalo: clasificacion.append(3)
  elif i <= 4*intervalo: clasificacion.append(4)
  elif i <= 5*intervalo: clasificacion.append(5)

arribo['clase']=clasificacion


arribo['Llegada']=pd.to_datetime(arribo['Llegada'],format='%H:%M:%S')

intervalos = [(6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22)]
es=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

def asignar_intervalo(hora):
    for i in range(len(intervalos)):
        if intervalos[i][0] <= hora.hour < intervalos[i][1]:
            return es[i]

arribo['intervalo'] = arribo['Llegada'].apply(asignar_intervalo)


In [169]:
meses_dict = {'Abril': 4, 'Septiembre': 9,'Noviembre': 11}
arribo['num_mes'] = arribo['Mes'].map(meses_dict)
servicios=list(arribo['TipoServ.'].unique())
servicio = {servicios[0]: 1, servicios[1]: 2,servicios[2]: 3,servicios[3]: 4,servicios[4]: 5}
arribo['tipo_serv'] = arribo['TipoServ.'].map(servicio)

#Hallando los kmeans, para determinar agrupamientos, se descubre que para el turno 1 se puede representar en 5 grupos y el turno 2, no arroja nada relevante

In [ ]:
kruskal,levene,pruebavar=pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
arreglo=[]
sorce=[]
turno=[]

for t in [1,2]:
  base=arribo[(arribo['Turno']==t)]
  base=base.reset_index()
  X = np.array(base['min_cliente']).reshape(-1, 1)
  df=pd.DataFrame()

  for k in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    df[k] = kmeans.labels_
  cosas=["dia","TipoServ.","intervalo","Cuadrilla"]

  for k in list(df.columns):
    base['kmeans']=df[k]

    for i in base['kmeans'].unique():
      eval=base[(base['kmeans']==i)]

      for j in cosas:
        mete=(k,i)
        sorce.append(j)
        turno.append(t)
        arreglo.append(mete)
        kruskal=pd.concat([kruskal,pg.kruskal(eval, dv='min_cliente', between=j)])
        levene=pd.concat([levene,pd.DataFrame(pg.homoscedasticity(eval, dv="min_cliente", group=j))])
        pruebavar=pd.concat([pruebavar,pg.homoscedasticity(eval, dv="min_cliente", group=j, method="bartlett", alpha=.05)])

kruskal['grupo']=arreglo
levene['grupo']=arreglo
pruebavar['grupo']=arreglo
kruskal['turno']=turno
levene['turno']=turno
pruebavar['turno']=turno
result=[]
for i in kruskal['p-unc']:
  if i<=5e-02:
    result.append(False)
  else:
    result.append(True)
kruskal["Independencia"]=result
levene['Source']=sorce


In [122]:
for t in [1,2]:
  print("**"*10)
  print("Para el turno "+str(t))
  for i in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
    print("%%%"*10)
    print("Para grupos de "+str(i))
    print("--"*10)
    print(kruskal[(kruskal['grupo'].apply(lambda x: x[0] == i))&(kruskal['Independencia']==False)&(kruskal['turno']==t)])
    print("++"*10)
    print(levene[(levene['grupo'].apply(lambda x: x[0] == i))&(levene['equal_var']==False)&(levene['turno']==t)])
    print("--"*10)

********************
Para el turno 1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Para grupos de 2
--------------------
            Source  ddof1          H     p-unc   grupo  turno  Independencia
Kruskal  intervalo      8  16.687837  0.033529  (2, 0)      1          False
++++++++++++++++++++
               W          pval  equal_var   grupo  turno     Source
levene  5.548208  7.984354e-07      False  (2, 0)      1  intervalo
--------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Para grupos de 3
--------------------
            Source  ddof1          H     p-unc   grupo  turno  Independencia
Kruskal  intervalo      7  19.860731  0.005878  (3, 1)      1          False
++++++++++++++++++++
               W          pval  equal_var   grupo  turno     Source
levene  2.714438  3.033439e-02      False  (3, 1)      1  TipoServ.
levene  7.532508  2.731065e-08      False  (3, 1)      1  intervalo
--------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Para grupos de 4
--------------------
            Source  ddof

#Realizando Gauss para determinar kmeans


In [175]:
cosas=["dia","TipoServ.","intervalo","Mes","Cuadrilla"]
kruskal,levene,pruebavar=pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
arreglo=[]
sorce=[]
turno=[]

for t in [1,2]:
  base=arribo[(arribo['Turno']==t)]
  base=base.reset_index()
  X = base[['min_cliente','num_mes','intervalo','tipo_serv','num_dia','dia_mes','Cuadrilla']]

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  df=pd.DataFrame()
  for n_components in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(X_scaled)

    # Obtener las etiquetas predichas
    labels = gmm.predict(X_scaled)

    # Agregar las etiquetas al DataFrame original
    df[n_components] = labels


  for k in list(df.columns):
    base['kmeans']=df[k]

    for i in base['kmeans'].unique():
      eval=base[(base['kmeans']==i)]

      for j in cosas:
        if len(eval[j].unique())>1:
          mete=(k,i)
          sorce.append(j)
          turno.append(t)
          arreglo.append(mete)
          kruskal=pd.concat([kruskal,pg.kruskal(eval, dv='min_cliente', between=j)])
          levene=pd.concat([levene,pd.DataFrame(pg.homoscedasticity(eval, dv="min_cliente", group=j))])
          pruebavar=pd.concat([pruebavar,pg.homoscedasticity(eval, dv="min_cliente", group=j, method="bartlett", alpha=.05)])

kruskal['grupo']=arreglo
levene['grupo']=arreglo
pruebavar['grupo']=arreglo
kruskal['turno']=turno
levene['turno']=turno
pruebavar['turno']=turno
kruskal["Independencia"] = kruskal['p-unc'] <= 0.05
levene['Source']=sorce

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:2902: RuntimeWarning: divide by zero encountered in divide
  denom = 1.0 + 1.0/(3*(k - 1)) * ((np.sum(1.0/(Ni - 1.0), axis=0)) -
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:2902: RuntimeWarning: div

In [176]:
for t in [1,2]:
  print("$$$$"*10)
  print("Para el turno "+str(t))
  for i in [2,3,4,5,6,7,8,9,10,11,12,13,14]:
    print("%%%"*10)
    print("Para grupos de "+str(i))
    print("--"*10)
    print(kruskal[(kruskal['grupo'].apply(lambda x: x[0] == i))&(kruskal['Independencia']==False)&(kruskal['turno']==t)])
    print("++"*10)
    print(levene[(levene['grupo'].apply(lambda x: x[0] == i))&(levene['equal_var']==False)&(levene['turno']==t)])
    print("--"*10)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Para el turno 1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Para grupos de 2
--------------------
            Source  ddof1         H     p-unc   grupo  turno  Independencia
Kruskal        dia      4  2.831563  0.586397  (2, 0)      1          False
Kruskal  TipoServ.      4  3.487359  0.479803  (2, 0)      1          False
Kruskal        Mes      2  0.120702  0.941434  (2, 0)      1          False
Kruskal  Cuadrilla      1  1.731680  0.188196  (2, 0)      1          False
Kruskal        dia      1  1.085045  0.297572  (2, 1)      1          False
Kruskal  TipoServ.      4  8.239910  0.083175  (2, 1)      1          False
Kruskal        Mes      2  5.580851  0.061395  (2, 1)      1          False
++++++++++++++++++++
               W          pval  equal_var   grupo  turno     Source
levene  6.611693  1.338105e-08      False  (2, 0)      1  intervalo
levene  4.424761  7.332107e-05      False  (2, 1)      1  intervalo
--------------------
%%%%%%%%%%%%%%%%%%%%